## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marzuq,YE,14.4000,46.4667,61.20,37,0,3.91,clear sky
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.12,50,72,26.22,broken clouds
2,2,Butaritari,KI,3.0707,172.7902,82.09,73,100,17.00,overcast clouds
3,3,Roma,US,43.2128,-75.4557,41.88,69,20,3.00,few clouds
4,4,Nioro,GM,13.3500,-15.7500,95.83,18,24,3.60,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria
prefferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
prefferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Butaritari,KI,3.0707,172.7902,82.09,73,100,17.00,overcast clouds
5,5,Mogadishu,SO,2.0371,45.3438,79.83,82,69,21.00,broken clouds
7,7,Araouane,ML,18.9048,-3.5265,80.29,7,9,7.31,clear sky
14,14,Hithadhoo,MV,-0.6000,73.0833,81.39,75,100,8.01,overcast clouds
16,16,Atuona,PF,-9.8000,-139.0333,78.75,77,20,18.90,few clouds
18,18,Kulhudhuffushi,MV,6.6221,73.0700,82.49,74,87,9.93,overcast clouds
19,19,Goias,BR,-15.5807,-49.6362,80.10,77,99,9.35,light rain
28,28,Beloha,MG,-25.1667,45.0500,76.71,74,72,11.50,broken clouds
35,35,Kapaa,US,22.0752,-159.3190,75.52,79,75,3.00,broken clouds
36,36,Avarua,CK,-21.2078,-159.7750,75.25,88,90,2.30,light rain


In [5]:
# 4a. Determine if there are any empty rows.
prefferred_cities_df.count()

City ID                206
City                   206
Country                205
Lat                    206
Lng                    206
Max Temp               206
Humidity               206
Cloudiness             206
Wind Speed             206
Current Description    206
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefferred_cities_df.dropna()
clean_df.count()

City ID                205
City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.09,overcast clouds,3.0707,172.7902,
5,Mogadishu,SO,79.83,broken clouds,2.0371,45.3438,
7,Araouane,ML,80.29,clear sky,18.9048,-3.5265,
14,Hithadhoo,MV,81.39,overcast clouds,-0.6000,73.0833,
16,Atuona,PF,78.75,few clouds,-9.8000,-139.0333,
18,Kulhudhuffushi,MV,82.49,overcast clouds,6.6221,73.0700,
19,Goias,BR,80.10,light rain,-15.5807,-49.6362,
28,Beloha,MG,76.71,broken clouds,-25.1667,45.0500,
35,Kapaa,US,75.52,broken clouds,22.0752,-159.3190,
36,Avarua,CK,75.25,light rain,-21.2078,-159.7750,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,82.09,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
5,Mogadishu,SO,79.83,broken clouds,2.0371,45.3438,Hotel Juba
14,Hithadhoo,MV,81.39,overcast clouds,-0.6000,73.0833,Scoop Guest House
16,Atuona,PF,78.75,few clouds,-9.8000,-139.0333,Villa Enata
18,Kulhudhuffushi,MV,82.49,overcast clouds,6.6221,73.0700,Haajy Guest House 1
35,Kapaa,US,75.52,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Avarua,CK,75.25,light rain,-21.2078,-159.7750,Paradise Inn
44,Chinsali,ZM,76.05,broken clouds,-10.5414,32.0816,Chisakatu Guest House
48,Iracoubo,GF,77.67,overcast clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
50,The Valley,AI,85.86,scattered clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weater</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))